### Data Exploration

This notebook explores the pre-processed data, and shows some basic statistics that may be useful.  

Before running this notebook, you will need to complete the setup steps in the README.md file located in the root of this repository.

In [50]:
import pandas as pd
from pathlib import Path
pd.set_option('max_colwidth',300)

### Reading The Data

The training data is located in `/resources/data`, which contains approximately 3.2 Million code, comment pairs across the train, validation, and test partitions.  You can learn more about the directory structure and associated files by viewing `/resources/README.md`.

The preprocessed data re stored in [json lines](http://jsonlines.org/) format.  First, we can get a list of all these files for further inspection:

In [86]:
pythonfiles = sorted(Path('../resources/data/python/').glob('**/*.gz'))
csharpfiles = sorted(Path('../resources/data/csharp/').glob('**/*.gz'))
javafiles = sorted(Path('../resources/data/java/').glob('**/*.gz'))
allfiles = pythonfiles + csharpfiles + javafiles

In [87]:
print(f'Total number of files: {len(allfiles):,}')

Total number of files: 3,192


To make analysis of this dataset easier, we can load all of the data into a pandas dataframe: 

In [80]:
columns_long_list = ['repo', 'path', 'lineno', 'code', 
                     'code_tokens', 'docstring', 'docstring_tokens', 
                     'language', 'partition']

columns_short_list = [ 'code_tokens', 'docstring_tokens', 
                        'language', 'partition']

def jsonl_list_to_dataframe(file_list, columns=columns_long_list):
    "Load a list of jsonl.gz files into a pandas DataFrame."
    return pd.concat([pd.read_json(f, 
                                   orient='records', 
                                   compression='gzip',
                                   lines=True)[columns] 
                      for f in file_list], sort=False)

This is what the python dataset looks like:

In [82]:
pydf = jsonl_list_to_dataframe(pythonfiles)

In [84]:
pydf.head(3)

,repo,path,lineno,code,code_tokens,docstring,docstring_tokens,language,partition
0,samluescher/django-media-tree,media_tree/templatetags/media_tree_tags.py,21,"def file_links(items, opts=None):\n """"""""""""\n result = []\n kwargs = get_kwargs_for_file_link(opts)\n for item in items:\n if isinstance(item, FileNode):\n result.append(get_file_link(item, **kwargs))\n else:\n result.append(file_links(item, kwargs)...","[def, file_links, items, opts, None, """""""""""", result, kwargs, get_kwargs_for_file_link, opts, for, item, in, items, if, isinstance, item, FileNode, result, append, get_file_link, item, kwargs, else, result, append, file_links, item, kwargs, return, result]","Turns a (optionally nested) list of FileNode objects into a list of \nstrings, linking to the associated files.","[turns, a, (, optionally, nested, ), list, of, filenode, objects, into, a, list, of, strings, linking, to, the, associated, files, .]",python,test
1,MediaKraken/MediaKraken_Deployment,source/common/common_metadata.py,28,"def com_meta_calc_trailer_weight(trailer_file_list, title_name, title_year):\n """"""""""""\n old_weight = 0\n best_match = None\n weight = 0\n for file_name in trailer_file_list:\n weight = 0\n if file_name.lower().find('official trailer') != -1:\n weight += 3\...","[def, com_meta_calc_trailer_weight, trailer_file_list, title_name, title_year, """""""""""", old_weight, best_match, None, weight, for, file_name, in, trailer_file_list, weight, if, file_name, lower, find, 'official trailer', weight, if, file_name, lower, find, title_name, lower, weight, if, file_name...","Determine ""weight"" of file to download for trailer","[determine, weight, of, file, to, download, for, trailer]",python,test
2,jptomo/ansible-connection-nsenter,nsenter.py,44,"def _get_container_env(self):\n """"""""""""\n env_path = '/proc/{}/environ'.format(self._extract_var('Leader'))\n env_str = self._exec_command('cat ' + env_path)[2]\n proc_envs = env_str.split('\x00')\n proc_envs = dict([x.split('=') for x in proc_envs if x])\n return proc_envs\n","[def, _get_container_env, self, """""""""""", env_path, '/proc/{}/environ', format, self, _extract_var, 'Leader', env_str, self, _exec_command, 'cat ', env_path, proc_envs, env_str, split, '\x00', proc_envs, dict, x, split, '=', for, x, in, proc_envs, if, x, return, proc_envs]",return container env dict,"[return, container, env, dict]",python,test


Two columns that will be heavily used in this dataset are `code_tokens` and `docstring_tokens`, which represent a parallel corpus that can be used for interesting tasks like information retrieval (for example trying to retrieve a codesnippet using the docstring.).  You can find more information regarding the definition of the above columns in the README of this repo. 

Next, we will read in all of the data for a limited subset of these columns into memory so we can compute summary statistics.  **Warning:** This step takes ~ 20 minutes.

In [88]:
alldf = jsonl_list_to_dataframe(allfiles, columns_short_list)

## Summary Statistics

### Row Counts

By Partition

In [90]:
alldf.partition.value_counts()

train    2060644
valid     519851
test      516239
Name: partition, dtype: int64

By Language

In [96]:
alldf.language.value_counts()

java      1808209
python     690299
csharp     598226
Name: language, dtype: int64

By Partition & Language

In [94]:
alldf.groupby(['partition', 'language'])['code_tokens'].count()

partition  language
test       csharp       100181
           java         301584
           python       114474
train      csharp       396736
           java        1203977
           python       459931
valid      csharp       101309
           java         302648
           python       115894
Name: code_tokens, dtype: int64

### Token Lengths By Language

In [97]:
alldf['code_len'] = alldf.code_tokens.apply(lambda x: len(x))
alldf['query_len'] = alldf.docstring_tokens.apply(lambda x: len(x))

#### Code Length Percentile By Language

For example, the 80th percentile length for python tokens is 72

In [98]:
code_len_summary = alldf.groupby('language')['code_len'].quantile([.5, .7, .8, .9, .95])
display(pd.DataFrame(code_len_summary))

code_len
language               
csharp   0.50      77.0
         0.70     120.0
         0.80     164.0
         0.90     273.0
         0.95     479.0
java     0.50      76.0
         0.70     118.0
         0.80     158.0
         0.90     240.0
         0.95     347.0
python   0.50      36.0
         0.70      55.0
         0.80      72.0
         0.90     106.0
         0.95     147.0

#### Query Length Percentile By Language

For example, the 80th percentile length for python tokens is 19

In [99]:
query_len_summary = alldf.groupby('language')['query_len'].quantile([.5, .7, .8, .9, .95])
display(pd.DataFrame(query_len_summary))

query_len
language                
csharp   0.50        8.0
         0.70       12.0
         0.80       16.0
         0.90       20.0
         0.95       27.0
java     0.50       11.0
         0.70       16.0
         0.80       22.0
         0.90       34.0
         0.95       48.0
python   0.50       10.0
         0.70       14.0
         0.80       19.0
         0.90       29.0
         0.95       41.0

#### Query Length All Languages

In [100]:
query_len_summary = alldf['query_len'].quantile([.5, .7, .8, .9, .95])
display(pd.DataFrame(query_len_summary))

,query_len
0.50,10.0
0.70,15.0
0.80,20.0
0.90,30.0
0.95,43.0
